# Data Creation from Food101 Database

creating a PyTorch dataset with differnt types of food. PyTorch now incorporates Food101.

Three classes: Pizza, Steak, Sushi.

The target dataset folder structure is as follows:

```
pizza_steak_sushi/
    train/
        pizza/
            image01.jpeg
            image02.jpeg
            ...
        steak/
            image04.jpeg
            image05.jpeg
            ...
        sushi/
            image07.jpeg
            ...
    test/
        pizza/
            image101.jpeg
            image102.jpeg
            ...
        steak/
            image104.jpeg
            image105.jpeg
            ...
        sushi/
            image107.jpeg
            ...
```

In [1]:
import torch
import torchvision

import torchvision.datasets as datasets
import torchvision.transforms as transforms



## Download data

Get the Food101 dataset from PyTorch.
* Food101 in `torchvision.datasets` - https://pytorch.org/vision/stable/generated/torchvision.datasets.Food101.html
* Original Food101 dataset - https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/ 

In [2]:
# Setup data directory
from pathlib import Path
data_dir = Path("../data")

# Get training data
train_data = datasets.Food101(root=data_dir,
                              split="train",
                              # transform=transforms.ToTensor(),
                              download=True)

# Get testing data
test_data = datasets.Food101(root=data_dir,
                             split="test",
                             # transform=transforms.ToTensor(),
                             download=True)

In [3]:
train_data

Dataset Food101
    Number of datapoints: 75750
    Root location: ..\data
    split=train

In [4]:
class_names = train_data.classes
class_names[:10]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito']

## Extract the Subset of Target Classes

A list of the different target image classes (`pizza`, `steak`, `sushi`) filenames will be created and copied to separate folders.

In [5]:
# Get 100% of training images
import random

# Setup data paths
data_path = data_dir / "food-101" / "images"
target_classes = ["pizza", "steak", "sushi"]

amount_to_get = 1.0

# Create function to separate a random amount of data
def get_subset(image_path=data_path,
               data_splits=["train", "test"], 
               target_classes=["pizza", "steak", "sushi", "hamburger", "apple_pie"],
               amount=0.1,
               seed=42):
    random.seed(42)
    label_splits = {}
    
    # Get labels
    for data_split in data_splits:
        print(f"[INFO] Creating image split for: {data_split}...")
        label_path = data_dir / "food-101" / "meta" / f"{data_split}.txt"
        with open(label_path, "r") as f:
            labels = [line.strip("\n") for line in f.readlines() if line.split("/")[0] in target_classes] 
        
        # Get random subset of target classes image ID's
        number_to_sample = round(amount * len(labels))
        print(f"[INFO] Getting random subset of {number_to_sample} images for {data_split}...")
        sampled_images = random.sample(labels, k=number_to_sample)
        
        # Apply full paths
        image_paths = [pathlib.Path(str(image_path / sample_image) + ".jpg") for sample_image in sampled_images]
        label_splits[data_split] = image_paths
    return label_splits
        
label_splits = get_subset(amount=amount_to_get)
label_splits["train"][:10]

[INFO] Creating image split for: train...
[INFO] Getting random subset of 3750 images for train...


NameError: name 'pathlib' is not defined

## Move training and testing images to dedicated folders

In [ ]:
# Create target directory path
target_dir_name = f"../data/pizza_steak_sushi_applepie_hamburger_{str(int(amount_to_get*100))}_percent"
print(f"Creating directory: '{target_dir_name}'")

# Setup the directories
target_dir = Path(target_dir_name)

# Make the directories
target_dir.mkdir(parents=True, exist_ok=True)

Creating directory: '../data/pizza_steak_sushi_applepie_hamburger_100_percent'


In [ ]:
import shutil

for image_split in label_splits.keys():
    for image_path in label_splits[str(image_split)]:
        dest_dir = target_dir / image_split / image_path.parent.stem / image_path.name
        if not dest_dir.parent.is_dir():
            dest_dir.parent.mkdir(parents=True, exist_ok=True)
        print(f"[INFO] Copying {image_path} to {dest_dir}...")
        shutil.copy2(image_path, dest_dir)

[INFO] Copying ..\data\food-101\images\steak\2796102.jpg to ..\data\pizza_steak_sushi_applepie_hamburger_100_percent\train\steak\2796102.jpg...
[INFO] Copying ..\data\food-101\images\apple_pie\3047214.jpg to ..\data\pizza_steak_sushi_applepie_hamburger_100_percent\train\apple_pie\3047214.jpg...
[INFO] Copying ..\data\food-101\images\apple_pie\1464079.jpg to ..\data\pizza_steak_sushi_applepie_hamburger_100_percent\train\apple_pie\1464079.jpg...
[INFO] Copying ..\data\food-101\images\sushi\1165660.jpg to ..\data\pizza_steak_sushi_applepie_hamburger_100_percent\train\sushi\1165660.jpg...
[INFO] Copying ..\data\food-101\images\hamburger\2877643.jpg to ..\data\pizza_steak_sushi_applepie_hamburger_100_percent\train\hamburger\2877643.jpg...
[INFO] Copying ..\data\food-101\images\hamburger\2201071.jpg to ..\data\pizza_steak_sushi_applepie_hamburger_100_percent\train\hamburger\2201071.jpg...
[INFO] Copying ..\data\food-101\images\hamburger\177861.jpg to ..\data\pizza_steak_sushi_applepie_hambur

In [ ]:
# Check lengths of directories
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  import os
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")
    
walk_through_dir(target_dir)

There are 2 directories and 0 images in '..\data\pizza_steak_sushi_applepie_hamburger_100_percent'.
There are 5 directories and 0 images in '..\data\pizza_steak_sushi_applepie_hamburger_100_percent\test'.
There are 0 directories and 250 images in '..\data\pizza_steak_sushi_applepie_hamburger_100_percent\test\apple_pie'.
There are 0 directories and 250 images in '..\data\pizza_steak_sushi_applepie_hamburger_100_percent\test\hamburger'.
There are 0 directories and 250 images in '..\data\pizza_steak_sushi_applepie_hamburger_100_percent\test\pizza'.
There are 0 directories and 250 images in '..\data\pizza_steak_sushi_applepie_hamburger_100_percent\test\steak'.
There are 0 directories and 250 images in '..\data\pizza_steak_sushi_applepie_hamburger_100_percent\test\sushi'.
There are 5 directories and 0 images in '..\data\pizza_steak_sushi_applepie_hamburger_100_percent\train'.
There are 0 directories and 750 images in '..\data\pizza_steak_sushi_applepie_hamburger_100_percent\train\apple_pie'

There is a total of 750 images per class for training and 250 images for testing.